In [5]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_weiwei.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [6]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,alpha,folds):
        #kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            if len(candi_fea)==optim_featur_num:
                return candi_fea
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                #print(len(candi_fea_copy))
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            total_score=alpha*relief_score+(1-alpha)*np.array(rfe_score)
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(i)
            i-=1

In [7]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,0.2,0.1)

In [8]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.7799859074064409, 0.7728430502635838, 0.7733532543452164, 0.772331541312177, 0.7682420794404718, 0.7723289315726289, 0.770799624197505, 0.7661960436348453, 0.7677383997077092, 0.7651821598204499, 0.7611005271673887, 0.7590610157106321, 0.7585547262383214, 0.7580497416357848, 0.7509003601440576, 0.7514131739652383, 0.7498838665901143, 0.7468148128816744, 0.7447739965551438, 0.7412012631139413, 0.7340557962315362, 0.718219896654314, 0.7080053760634689, 0.7100435826504514, 0.6967600083511666, 0.6926692416096873, 0.6804099900829896, 0.6691672321102354, 0.6819249438905997, 0.6788637194008038, 0.6701785061850828, 0.6630382587817737, 0.6574194895349444, 0.6512944308158046, 0.6477230022443761, 0.6426209614280495, 0.6385393287749883, 0.635988308366825, 0.6324168797953964, 0.6283352471423351, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0.

In [9]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.7799859074064409, 0.7728430502635838, 0.7733532543452164, 0.772331541312177, 0.7733493397358944, 0.7687470640430084, 0.7764144788350122, 0.7723250169633071, 0.7677383997077092, 0.7651821598204499, 0.7611005271673887, 0.7590610157106321, 0.7585547262383214, 0.7580497416357848, 0.7509003601440576, 0.7514131739652383, 0.7498838665901143, 0.7468148128816744, 0.7447739965551438, 0.7412012631139413, 0.7340557962315362, 0.718219896654314, 0.7080053760634689, 0.7100435826504514, 0.6967600083511666, 0.6926692416096873, 0.6804099900829896, 0.6691672321102354, 0.6819249438905997, 0.6788637194008038, 0.6701785061850828, 0.6630382587817737, 0.6574194895349444, 0.6512944308158046, 0.6477230022443761, 0.6426209614280495, 0.6385393287749883, 0.635988308366825, 0.6324168797953964, 0.6283352471423351, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0

In [10]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.7799859074064409, 0.7728430502635838, 0.7733532543452164, 0.772331541312177, 0.7733493397358944, 0.7687470640430084, 0.7764144788350122, 0.7723250169633071, 0.7677383997077092, 0.7651821598204499, 0.7611005271673887, 0.7590610157106321, 0.7585547262383214, 0.7580497416357848, 0.7509003601440576, 0.7514131739652383, 0.7498838665901143, 0.7468148128816744, 0.7447739965551438, 0.7412012631139413, 0.7340557962315362, 0.718219896654314, 0.7080053760634689, 0.6977856359935278, 0.6967600083511666, 0.6926692416096873, 0.6804099900829896, 0.6691672321102354, 0.6819249438905997, 0.6788637194008038, 0.6701785061850828, 0.6630382587817737, 0.6574194895349444, 0.6512944308158046, 0.6477230022443761, 0.6426209614280495, 0.6385393287749883, 0.635988308366825, 0.6324168797953964, 0.6283352471423351, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0

In [11]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.7799859074064409, 0.7728430502635838, 0.7733532543452164, 0.772331541312177, 0.7733493397358944, 0.7687470640430084, 0.7764144788350122, 0.7723250169633071, 0.7677383997077092, 0.7651821598204499, 0.7611005271673887, 0.7590610157106321, 0.7585547262383214, 0.7580497416357848, 0.7509003601440576, 0.7514131739652383, 0.7498838665901143, 0.7468148128816744, 0.7447739965551438, 0.7412012631139413, 0.7340557962315362, 0.718219896654314, 0.7080053760634689, 0.6977856359935278, 0.6967600083511666, 0.6926692416096873, 0.6804099900829896, 0.6691672321102354, 0.6819249438905997, 0.6788637194008038, 0.6701785061850828, 0.6630382587817737, 0.6574194895349444, 0.6512944308158046, 0.6477230022443761, 0.6426209614280495, 0.6385393287749883, 0.635988308366825, 0.6324168797953964, 0.6283352471423351, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0

In [12]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.7799859074064409, 0.7728430502635838, 0.7733532543452164, 0.772331541312177, 0.7733493397358944, 0.7687470640430084, 0.7764144788350122, 0.7723250169633071, 0.7677383997077092, 0.7651821598204499, 0.7611005271673887, 0.7590610157106321, 0.7585547262383214, 0.7580497416357848, 0.7509003601440576, 0.7514131739652383, 0.7498838665901143, 0.7468148128816744, 0.7447739965551438, 0.7412012631139413, 0.7340557962315362, 0.718219896654314, 0.7080053760634689, 0.6977856359935278, 0.6967600083511666, 0.6926692416096873, 0.6804099900829896, 0.6691672321102354, 0.6819249438905997, 0.6788637194008038, 0.6701785061850828, 0.6630382587817737, 0.6574194895349444, 0.6512944308158046, 0.6477230022443761, 0.6426209614280495, 0.6385393287749883, 0.635988308366825, 0.6324168797953964, 0.6283352471423351, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0

In [13]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.7799859074064409, 0.7728430502635838, 0.7733532543452164, 0.772331541312177, 0.7733493397358944, 0.7687470640430084, 0.7764144788350122, 0.7723250169633071, 0.7677383997077092, 0.7651821598204499, 0.7611005271673887, 0.7590610157106321, 0.7585547262383214, 0.7580497416357848, 0.7509003601440576, 0.7514131739652383, 0.7498838665901143, 0.7468148128816744, 0.7447739965551438, 0.7412012631139413, 0.7340557962315362, 0.718219896654314, 0.7080053760634689, 0.6977856359935278, 0.6967600083511666, 0.6855133357690902, 0.6717169476486247, 0.6885589018215982, 0.6819249438905997, 0.6788637194008038, 0.6701785061850828, 0.6630382587817737, 0.6574194895349444, 0.6512944308158046, 0.6477230022443761, 0.6426209614280495, 0.6385393287749883, 0.635988308366825, 0.6324168797953964, 0.6283352471423351, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0

In [14]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.7799859074064409, 0.7830484367660108, 0.7799872122762148, 0.7764131739652382, 0.7718135080119005, 0.7758977504045096, 0.7723289315726289, 0.766704942846704, 0.7733545592149903, 0.7713124380186859, 0.7702920298554204, 0.773871287645493, 0.7651978182577379, 0.7606033717834959, 0.7595842684900046, 0.7539681089827235, 0.7534592097708649, 0.7478300015658437, 0.737109191502688, 0.7268998903909389, 0.7171955738817266, 0.7120857038467561, 0.7115715851558015, 0.7039133044522157, 0.6916514431859702, 0.6906245106738347, 0.6875580667049428, 0.6737616785844773, 0.684475964298763, 0.6788598047914818, 0.6747742575290986, 0.6686478939401848, 0.6620126311394123, 0.6548645545174592, 0.6461910851297041, 0.6410890443133775, 0.6400686361501122, 0.6354781042851924, 0.6324168797953964, 0.6278224333211545, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6074155749256225, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0.

In [15]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.7799859074064409, 0.7830484367660108, 0.7799872122762148, 0.7764131739652382, 0.7718135080119005, 0.7758977504045096, 0.7723289315726289, 0.766704942846704, 0.7733545592149903, 0.7713124380186859, 0.7636541573151, 0.7626441881100267, 0.7631530873218852, 0.7606033717834959, 0.7595842684900046, 0.7539681089827235, 0.7534592097708649, 0.7478300015658437, 0.737109191502688, 0.7268998903909389, 0.7171955738817266, 0.7120857038467561, 0.7115715851558015, 0.7039133044522157, 0.6916514431859702, 0.6906245106738347, 0.6875580667049428, 0.6737616785844773, 0.684475964298763, 0.6788598047914818, 0.6747742575290986, 0.6686478939401848, 0.6620126311394123, 0.65384284148442, 0.6461910851297041, 0.6456808810480714, 0.6375176157419489, 0.6339474920402944, 0.6324168797953964, 0.6278224333211545, 0.6242523096194998, 0.6196604728848062, 0.6160890443133775, 0.6145584320684796, 0.6104767994154183, 0.6089461871705204, 0.6048645545174592, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0.8051

In [16]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.7799859074064409, 0.7830484367660108, 0.7799859074064408, 0.7769207683073229, 0.7774322772587297, 0.7779385667310403, 0.773853019468657, 0.7794704838457122, 0.7713059136698158, 0.7718226421003184, 0.7743775771178036, 0.7702959444647425, 0.7672360248447205, 0.7641695808758285, 0.7626402735007047, 0.7529359569914922, 0.7483336812986063, 0.7570136750352315, 0.7519090244793569, 0.7488451902500131, 0.738620230700976, 0.7360666005532648, 0.7289185239313116, 0.7222806513909912, 0.7100200949945196, 0.6957148076621953, 0.6936739913356647, 0.6885628164309201, 0.6834516415261758, 0.682938827704995, 0.6732436452842007, 0.6691594028915914, 0.6563938618925832, 0.6512931259460306, 0.6482319014562347, 0.6446604728848062, 0.6385380239052141, 0.6385380239052143, 0.6349665953337856, 0.6319040659742157, 0.625781616994624, 0.6237408006680933, 0.6222114932929694, 0.6171107573464168, 0.6166005532647842, 0.6130278198235816, 0.6079257790072552, 0.6043543504358265, 0.6038441463541938] [0.8044223885313594, 0.

In [35]:
np.max(aver_mean_lis1[:10]),np.max(aver_mean_lis2[:10]),np.max(aver_mean_lis3[:10]),np.max(aver_mean_lis4[:10]),np.max(aver_mean_lis5[:10])

(0.7799859074064409,
 0.7799859074064409,
 0.7799859074064409,
 0.7799859074064409,
 0.7799859074064409)

In [36]:
np.max(aver_mean_lis6[:10]),np.max(aver_mean_lis7[:10]),np.max(aver_mean_lis8[:10]),np.max(aver_mean_lis9[:10])

(0.7799859074064409,
 0.7830484367660108,
 0.7830484367660108,
 0.7830484367660108)

In [38]:
np.argmax(aver_mean_lis7[:10]),np.argmax(aver_mean_lis8[:10]),np.argmax(aver_mean_lis9[:10])

(1, 1, 1)

# SVM-RFE

In [18]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,c,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=c
        self.Gamma=gamma
    def featu_score(self,folds):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[j for j in range(featu_num)]
        i=featu_num
        aver_lis=[j for j in range(featu_num)]
        F_lis=[j for j in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                rfe_score[k]=scor_fea
                k=k+1     
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=folds,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            if len(candi_fea)==featur_num:
                return candi_fea
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            suppor_index=estimator.support_
            support_num=len(suppor_index)
            support_vector=estimator.support_vectors_
            dual_coef=estimator.dual_coef_[0]
            rfe_score=[i for i in range(len(candi_fea))]
            z=np.exp(-self.Gamma*np.sum(support_vector*support_vector,axis=1))
            matrix_z=np.outer(z, z)
            y_matrix=np.outer(self.Y_train[suppor_index],self.Y_train[suppor_index])
            H_matrix=np.exp(2*self.Gamma*support_vector@support_vector.T)*matrix_z*y_matrix
            orig_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix),dual_coef*self.Y_train[suppor_index])
            k=0
            for fea in candi_fea:
                copy_support_vector=self.X_train[suppor_index,:]
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(fea)
                removed_support_vector=copy_support_vector[:,candi_fea_copy]
                z1=np.exp(-self.Gamma*np.sum(removed_support_vector*removed_support_vector,axis=1))
                matrix_z1=np.outer(z1, z1)
                H_matrix1=np.exp(2*self.Gamma*removed_support_vector@removed_support_vector.T)*matrix_z1*y_matrix
                after_loss=np.dot(np.dot(dual_coef*self.Y_train[suppor_index],H_matrix1),dual_coef*self.Y_train[suppor_index])
                scor_fea=np.abs(orig_loss-after_loss)/2
                #print(scor_fea)
                rfe_score[k]=scor_fea
                k=k+1     
            #print(rfe_score)
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            print(1)

In [19]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.2,0.1)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [20]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.7799859074064409,
  0.7728430502635838,
  0.7733532543452164,
  0.772331541312177,
  0.7682420794404718,
  0.7723289315726289,
  0.770799624197505,
  0.7661960436348453,
  0.7677383997077092,
  0.7651821598204499,
  0.7611005271673887,
  0.7590610157106321,
  0.7585547262383214,
  0.7580497416357848,
  0.7509003601440576,
  0.7514131739652383,
  0.7498838665901143,
  0.7468148128816744,
  0.7447739965551438,
  0.7412012631139413,
  0.7340557962315362,
  0.718219896654314,
  0.7080053760634689,
  0.7100435826504514,
  0.6967600083511666,
  0.6926692416096873,
  0.6804099900829896,
  0.6691672321102354,
  0.6819249438905997,
  0.6788637194008038,
  0.6701785061850828,
  0.6630382587817737,
  0.6574194895349444,
  0.6512944308158046,
  0.6477230022443761,
  0.6426209614280495,
  0.6385393287749883,
  0.635988308366825,
  0.6324168797953964,
  0.6283352471423351,
  0.6242523096194998,
  0.6196604728848062,
  0.6160890443133775,
  0.6145584320684796,
  0.6104767994154183,
  0.6089461871

# SVM-RELIEF

In [21]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C,gamma):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
        self.Gamma=gamma
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(C=self.C, kernel='rbf',gamma=self.Gamma)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [22]:
relief_SVM_model=relief_SVM(x_train,y_train,0.2,0.1)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [23]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.7789654992431755,
  0.788663291403518,
  0.7886659011430659,
  0.7830458270264626,
  0.7871313742888459,
  0.7866224750769872,
  0.7856020669137219,
  0.7922360248447206,
  0.7937692468291664,
  0.7922373297144945,
  0.7907054125998225,
  0.7901978182577378,
  0.7901952085181898,
  0.7876389686309306,
  0.7810037058301581,
  0.7799872122762148,
  0.7789641943734015,
  0.7810010960906102,
  0.7799885171459888,
  0.7815178245211127,
  0.7758990552742836,
  0.7738595438175271,
  0.7620883135863041,
  0.7539198288010857,
  0.7554530507855315,
  0.7523892165561876,
  0.7529059449866903,
  0.7401234406806201,
  0.7355263844668303,
  0.7258103241296519,
  0.7201967743619188,
  0.7166227360509421,
  0.7115206952346156,
  0.7013100892530926,
  0.697734746072342,
  0.6936505036797327,
  0.6880369539119995,
  0.682933608225899,
  0.6757855316039459,
  0.6706821859178455,
  0.6686387598517669,
  0.6660877394436036,
  0.6574116603163004,
  0.6517981105485673,
  0.6512866015971606,
  0.639551907

In [43]:
weiwei_accuracy=[]
weiwei_accuracy.append(aver_mean_lis1)
weiwei_accuracy.append(aver_mean_lis2)
weiwei_accuracy.append(aver_mean_lis3)
weiwei_accuracy.append(aver_mean_lis4)
weiwei_accuracy.append(aver_mean_lis5)
weiwei_accuracy.append(aver_mean_lis6)
weiwei_accuracy.append(aver_mean_lis7)
weiwei_accuracy.append(aver_mean_lis8)
weiwei_accuracy.append(aver_mean_lis9)
weiwei_accuracy.append(rfe_SVM_aver_mean_lis)
weiwei_accuracy.append(relief_SVM_aver_mean_lis)

In [44]:
colu=[i for i in range(1,50)]
weiwei_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(weiwei_accuracy))
weiwei_acurracy_dataframe.to_csv("weiwei_accuracy.csv")

In [45]:
weiwei_F_measure=[]
weiwei_F_measure.append(F_measure_lis1)
weiwei_F_measure.append(F_measure_lis2)
weiwei_F_measure.append(F_measure_lis3)
weiwei_F_measure.append(F_measure_lis4)
weiwei_F_measure.append(F_measure_lis5)
weiwei_F_measure.append(F_measure_lis6)
weiwei_F_measure.append(F_measure_lis7)
weiwei_F_measure.append(F_measure_lis8)
weiwei_F_measure.append(F_measure_lis9)
weiwei_F_measure.append(rfe_SVM_F_measure_lis)
weiwei_F_measure.append(relief_SVM_F_measure_lis)

In [46]:
colu=[i for i in range(1,50)]
weiwei_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(weiwei_F_measure))
weiwei_F_measure_dataframe.to_csv("weiwei_F_measure.csv")